In [6]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-malaysian-rag/resolve/main/rag-multiturn-chaotic.jsonl
# !wget https://huggingface.co/datasets/mesolitica/mixtral-malaysian-rag/resolve/main/rag-multiturn-chaotic-part2.jsonl

In [7]:
!head -n 1 rag-multiturn-chaotic-part2.jsonl

{"filename": "gov.my.jsonl", "chat": [{"role": "user", "content": "Are you as sure as a horse's hind leg?", "content_ms": "Adakah anda pasti seperti kaki belakang kuda?"}, {"role": "assistant", "content": "I'm afraid I don't understand the reference to a horse's hind leg in relation to my ability to provide accurate information. However, I can assure you that I will provide information based solely on the knowledge base you've given me and to the best of my ability, I will provide accurate and relevant responses. I don't have feelings, beliefs, or personal knowledge, I am a machine learning model that generate text based on the input data.", "content_ms": "Saya takut saya tidak memahami rujukan kepada kaki belakang kuda berkaitan dengan keupayaan saya untuk memberikan maklumat yang tepat. Walau bagaimanapun, saya boleh memberi jaminan kepada anda bahawa saya akan memberikan maklumat hanya berdasarkan pangkalan pengetahuan yang anda berikan kepada saya dan setakat kemampuan saya, saya a

In [8]:
import json
import random
from tqdm import tqdm
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [9]:
data = []
files = ['rag-multiturn-chaotic-part2.jsonl', 'rag-multiturn-chaotic.jsonl']
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            data.append(l)

169403it [00:08, 20529.86it/s]
327980it [00:16, 19882.92it/s]


In [13]:
data = [(i, data[i]) for i in range(len(data))]

In [18]:
ranged = set(range(len(data)))

In [24]:
!rm -rf mining-mixtral-malaysian-rag-pair
!mkdir mining-mixtral-malaysian-rag-pair

In [25]:
import os

def loop(rows):
    rows, index = rows
    for i in tqdm(range(len(rows))):
        filename = os.path.join('mining-mixtral-malaysian-rag-pair', f'{i}-{index}.json')
        if os.path.exists(filename):
            continue
            
        no, d = rows[i]
        
        pairs = []
        chat = {}
        list_of_groups = zip(*(iter(d['chat']),) * 2)
        for c in list_of_groups:
            chat[c[0]['content']] = c[1]['content']
            
        questions = d['questions']

        for q in questions:
            pos = [d['context']]
            if q in chat:
                pos.append(chat[q])

            sampled = random.sample(list(ranged - {no}), 100)
            neg = []
            for s in sampled:
                overlapped = overlap(q, data[s][1]['context'])
                if overlapped < 0.1:
                    neg.append(data[s][1]['context'])
                if len(neg) >= 3:
                    break

            pairs.append(
                {
                    'query': q,
                    'pos': pos,
                    'neg': neg,
                }
            )
        
        with open(filename, 'w') as fopen:
            json.dump(pairs, fopen)

In [26]:
loop((data[:100],0))

100%|██████████| 100/100 [00:10<00:00,  9.30it/s]


In [ ]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [36]:
import mp

mp.multiprocessing(data, loop, cores = 20, returned = False)

  5%|▍         | 1155/24869 [10:58<4:09:46,  1.58it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 19%|█▊        | 4614/24869 [49:27<3:19:25,  1.69it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 24%|██▍       | 6033/24869 [1:07:23<3:11:44,  1.64it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Noteb

In [37]:
from glob import glob

In [38]:
files = glob('mining-mixtral-malaysian-rag-pair/*.json')
len(files)

497383

In [40]:
!du -hs mining-mixtral-malaysian-rag-pair

105G	mining-mixtral-malaysian-rag-pair


In [43]:
import msgspec

In [44]:
with open('mixtral-malaysian-rag-pair.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            d = json.load(fopen)
        for d_ in d:
            fopen_l.write(f'{msgspec.json.encode(d_).decode()}\n')

100%|██████████| 497383/497383 [33:27<00:00, 247.73it/s]  


In [45]:
!ls -lh mixtral-malaysian-rag-pair.jsonl

-rw-r--r-- 1 ubuntu ubuntu 102G Mar  6 15:55 mixtral-malaysian-rag-pair.jsonl


In [49]:
!head -n 3 mixtral-malaysian-rag-pair.jsonl

{"query":"What did Proton state regarding under declaration of imported cars?","pos":["tersebut, pihak Proton menyatakan bahawa under declaration berlaku tetapi ini tidak diterjemahkan dalam bentuk harga pasaran kerana kalau kita hendak mengambil tindakan di bawah Anti Lambakan, under declaration ini mesti diterjemahkan dalam bentuk harga yang rendah, barulah berlaku injury, kecederaan kepada industri, baru tindakan boleh diambil. Daripada apa yang kita lakukan berteraskan survey kita, kereta-kereta yang diimport tersebut adalah lebih mahal, Yang kategori sama dengan Proton lebih mahal daripada Proton daripada RM6,000 sebuah kepada RM18,000 sebuah. Yang Berhormat menyatakan tentang kesan kepada Proton dari segi kemasukan kereta import ini. Sebenarnya kesan ini ada datang daripada dua sudut yang besar, satu dari segi dalaman dan satu dari segi luaran. Dari luaran ini ada 4 sudut, Tuan Yang di-Pertua. Satu, saya setuju Yang Berhormat kata kereta AP ini ada kesan tetapi kalau kita lihat d

In [48]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         124G   90G   35G  72% /
tmpfs            64M     0   64M   0% /dev
/dev/sdc        984G  612G  372G  63% /home/ubuntu
/dev/sdd        9.8G  131M  9.7G   2% /dev/shm
/dev/root       124G   90G   35G  72% /etc/hosts
tmpfs           205G   12K  205G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           109G   12K  109G   1% /proc/driver/nvidia
tmpfs            44G  185M   44G   1% /run/nvidia-persistenced/socket
tmpfs           109G     0  109G   0% /proc/acpi
tmpfs           109G     0  109G   0% /proc/scsi
tmpfs           109G     0  109G   0% /sys/firmware


In [51]:
!split -b 10G -d --additional-suffix=.splitted mixtral-malaysian-rag-pair.jsonl mixtral-malaysian-rag-pair.jsonl

In [55]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
for f in glob('mixtral-malaysian-rag-pair.jsonl*.splitted'):
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/instructions-pair-mining',
        repo_type='dataset',
)